In [2]:
import pymysql
import pandas as pd
import math
import yfinance as yf
import numpy as np
from nsepy.derivatives import get_expiry_date
from nsepy import get_history
from dateutil.relativedelta import relativedelta, TH
from datetime import date, timedelta
from nsepy.derivatives import get_expiry_date

In [3]:
comp_list = ['ADANIPORTS', 'ASIANPAINT', 'AUBANK', 'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJFINANCE', 'BANDHANBNK', 'BHARTIARTL', 'BPCL', 'BRITANNIA', 'CIPLA', 'COALINDIA', 'DIVISLAB', 'DRREDDY', 'EICHERMOT', 'FEDERALBNK', 'GRASIM', 'HCLTECH', 'HDFC', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO', 'HINDUNILVR', 'ICICIBANK', 'IDFCFIRSTB', 'INDUSINDBK', 'INFY', 'IOC', 'ITC', 'JSWSTEEL', 'KOTAKBANK', 'LT', 'M&M', 'MARUTI', 'NESTLEIND', 'NTPC', 'ONGC', 'PNB', 'POWERGRID', 'RBLBANK', 'RELIANCE', 'SBILIFE', 'SBIN', 'SHREECEM', 'SUNPHARMA', 'TATACONSUM', 'TATAMOTORS', 'TATASTEEL', 'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'UPL', 'WIPRO']


In [4]:
def get_last_thurs(year,month):
    dt= date(year,month ,1)
    cmon = month 
    t=dt
    for i in range(1, 6):
        t = dt + relativedelta(weekday=TH(i))
        if t.month != cmon:
        # since t is exceeded we need last one  which we can get by subtracting -2 since it is already a Thursday.
            t = t + relativedelta(weekday=TH(-2))
            break
    return t

In [5]:
dict_rvol={}
for comp in comp_list:
    dict_rvol[comp]=0

In [6]:
ZZZ= []

In [9]:
today = date.today()
cmon = today.month

for i in range(1, 6):
    t = today + relativedelta(weekday=TH(i))
    if t.month != cmon:
        # since t is exceeded we need last one  which we can get by subtracting -2 since it is already a Thursday.
        t = t + relativedelta(weekday=TH(-2))
        break
t

datetime.date(2022, 3, 31)

In [8]:
days_to_expiry = t.day -today.day

#days_to_expiry
days_to_expiry

23

In [ ]:
print('expiry on : ',end='')
print(t)
if(t.day >today.day+7 ):
    print('expiry in this month')
    
    for i in range(1,today.month):
        #getting last thurs of that month
        thurs = get_last_thurs(year=today.year,month=i)
        print('expiry on '+ str(thurs))
        
        
        for comp in comp_list :
            print('getting data from '+ comp)
            comp_data = yf.download(comp + '.ns',
                               start = date(today.year  ,i,1), 
                               end = thurs,progress=False)
            close = np.array(comp_data['Close'])
            #print(close.shape[0])
          
          #  print(comp_data)
            days_considered =min(days_to_expiry,close.shape[0])
            close = close[-days_considered:]
#             print(days_considered)
            close = close[~np.isnan(close)]
#             print(close.shape[0])
#             print(close[-1])
            
            log = [np.log10(close[i]/close[i-1]) for i in range(1,len(close))]
#             print(len(log))
#             print('log and std')
#             print(log)
            std=np.std(log)
#             print(std)
            rvol = std* np.sqrt(252)
            dict_rvol[comp]+=rvol/12
            if(comp in dict_rvol):
               
                dict_rvol[comp]+=rvol/12
            else:
                dict_rvol[comp]=rvol/12
            
            
            print(str(dict_rvol[comp])+' '+ comp+' rvol ')
    
    
    
    for i in range(today.month,13):
        #getting last thurs of that month
#         set_thurs = get_expiry_date(year=today.year-1,month=i)
#         list_thurs = []
#         for _ in set_thurs:
#             list_thurs.append(_)
#         list_thurs.sort()
#         thurs=list_thurs[-1]
        thurs = get_last_thurs(year=today.year-1,month=i)
        print('expiry on '+ str(thurs))
        
        
        for comp in comp_list :
            print('getting data from '+ comp)
            comp_data = yf.download(comp + '.ns',
                               start = date(today.year -1  ,i,1), 
                               end = thurs,progress=False)
            close = np.array(comp_data['Close'])
            #print(close.shape[0])
#             print(close[-1])
          
          #  print(comp_data)
            days_considered =min(days_to_expiry,close.shape[0])
            close = close[-days_considered:]
            close = close[~np.isnan(close)]
#             print(days_considered)
#             print(close.shape[0])
          
            log = [np.log10(close[i]/close[i-1]) for i in range(1,len(close))]
#             print('log and std')
#             print(log)
            std=np.std(log)
            rvol = std* np.sqrt(252)
#             print(std)
            if(comp in dict_rvol):
               
                dict_rvol[comp]+=rvol/12
            else:
                dict_rvol[comp]=rvol/12
            print(str(dict_rvol[comp])+' '+ comp+' rvol ')
            

elif( t.day<today.day):
    days_to_expiry=35
    print('expiry in next month')
    for i in range(1,today.month+1):
        #getting last thurs of that month
        thurs = get_last_thurs(year=today.year,month=i)
        print('expiry on '+ str(thurs))
        for comp in comp_list :
            print('getting data from '+ comp)
            comp_data = yf.download(comp + '.ns',
                               start = date(today.year,i,1), 
                               end = thurs,progress=False)
            close = np.array(comp_data['Close'])
            
            #print(close.shape[0])
             
          
          #  print(comp_data)
            days_considered =min(days_to_expiry,close.shape[0])
            close = close[-days_considered:]
#             print(days_considered)
#             print(close.shape[0])
            close = close[~np.isnan(close)]
            log = [np.log10(close[i]/close[i-1]) for i in range(1,len(close))]
#             print('log and std')
#             print(log)
#             print(len(log))
            std=np.std(log)
#             print(std)
            rvol = std* np.sqrt(252)
            
            
               
            if(comp in dict_rvol):
               
                dict_rvol[comp]+=rvol/12
            else:
                dict_rvol[comp]=rvol/12
            
            
            print(str(dict_rvol[comp])+' '+ comp+' rvol ')
    
           
    
    for i in range(today.month+1,13):
        #getting last thurs of that month
        thurs = get_last_thurs(year=today.year-1,month=i)
        
        print('expiry on '+ str(thurs))
        
        
        for comp in comp_list :
            print('getting data from '+ comp)
            comp_data = yf.download(comp + '.ns',
                               start = date(today.year -1  ,i,1), 
                               end = thurs,progress=False)
            close = np.array(comp_data['Close'])
            #print(close.shape[0])
            
          
          #  print(comp_data)
            days_considered =min(days_to_expiry,close.shape[0])
#             print(days_considered)
            close = close[-days_considered:]
            #print(days_considered)
#             print(close.shape[0])
          #np.log10(close/close.shift())
            close = close[~np.isnan(close)]
            log = [np.log10(close[i]/close[i-1]) for i in range(1,len(close))]
            
            std=np.std(log)
            rvol = std* np.sqrt(252)
            
            
            if(comp in dict_rvol):
               
                dict_rvol[comp]+=rvol/12
            else:
                dict_rvol[comp]=rvol/12
            
                
            print(str(dict_rvol[comp])+' '+ comp+' rvol ')
            

expiry on : 2022-03-31
expiry in this month
expiry on 2022-01-27
getting data from ADANIPORTS
0.019961128618496842 ADANIPORTS rvol 
getting data from ASIANPAINT
0.018457363616155537 ASIANPAINT rvol 
getting data from AUBANK
0.029532197382204903 AUBANK rvol 
getting data from AXISBANK
0.02412236850561145 AXISBANK rvol 
getting data from BAJAJ-AUTO
0.02061233317396274 BAJAJ-AUTO rvol 
getting data from BAJAJFINSV
0.02800360342829904 BAJAJFINSV rvol 
getting data from BAJFINANCE
0.025256256256769014 BAJFINANCE rvol 
getting data from BANDHANBNK
0.02284698934522429 BANDHANBNK rvol 
getting data from BHARTIARTL
0.01995827664902285 BHARTIARTL rvol 
getting data from BPCL
0.016523110416392497 BPCL rvol 
getting data from BRITANNIA
0.013230094190132059 BRITANNIA rvol 
getting data from CIPLA
0.014214373138055292 CIPLA rvol 
getting data from COALINDIA
0.021788190115244816 COALINDIA rvol 
getting data from DIVISLAB
0.01822309259740394 DIVISLAB rvol 
getting data from DRREDDY
0.01181245328887378

17
0.050152630215893876 HDFC rvol 
getting data from HDFCBANK
17
0.0422148131465024 HDFCBANK rvol 
getting data from HDFCLIFE
17
0.04906126981501869 HDFCLIFE rvol 
getting data from HEROMOTOCO
17
0.05090467124513623 HEROMOTOCO rvol 
getting data from HINDALCO
17
0.0567162371617451 HINDALCO rvol 
getting data from HINDUNILVR
17
0.03850360772270296 HINDUNILVR rvol 
getting data from ICICIBANK
17
0.042513808397020736 ICICIBANK rvol 
getting data from IDFCFIRSTB
17
0.058989299633970774 IDFCFIRSTB rvol 
getting data from INDUSINDBK
17
0.05794109113385881 INDUSINDBK rvol 
getting data from INFY
17
0.046572219418438685 INFY rvol 
getting data from IOC
17
0.0439821830858022 IOC rvol 
getting data from ITC
17
0.04104842953682952 ITC rvol 
getting data from JSWSTEEL
17
0.0674479381526225 JSWSTEEL rvol 
getting data from KOTAKBANK
17
0.05273417456257288 KOTAKBANK rvol 
getting data from LT
17
0.04809672483364376 LT rvol 
getting data from M&M
17
0.04790440129498223 M&M rvol 
getting data from MAR

In [ ]:
print(dict_rvol)

In [16]:
print(dict_rvol)

{'ADANIPORTS': 0.17732100783704988, 'ASIANPAINT': 0.12279651764222473, 'AUBANK': 0.19636290776743007, 'AXISBANK': 0.12851147937637314, 'BAJAJ-AUTO': 0.11637187648354819, 'BAJAJFINSV': 0.16165321439281685, 'BAJFINANCE': 0.1602820356051033, 'BANDHANBNK': 0.1695672699590855, 'BHARTIARTL': 0.1217762335823667, 'BPCL': 0.12271894149800208, 'BRITANNIA': 0.0919425395353547, 'CIPLA': 0.1081697016899526, 'COALINDIA': 0.15089614301418464, 'DIVISLAB': 0.12150058467924936, 'DRREDDY': 0.10647201111077245, 'EICHERMOT': 0.12762244218791846, 'FEDERALBNK': 0.1690738214715094, 'GRASIM': 0.13562788298784278, 'HCLTECH': 0.12308818445096797, 'HDFC': 0.12162523354078969, 'HDFCBANK': 0.10482670839166387, 'HDFCLIFE': 0.10827188227415156, 'HEROMOTOCO': 0.11148571050343038, 'HINDALCO': 0.17615380798165733, 'HINDUNILVR': 0.09551389706348137, 'ICICIBANK': 0.12199650268573597, 'IDFCFIRSTB': 0.17923898217381518, 'INDUSINDBK': 0.1704161521533902, 'INFY': 0.10414934168756024, 'IOC': 0.12309698107129928, 'ITC': 0.11550

comp_data

In [64]:
comp_data= np.array(comp_data['Close'])

In [65]:
comp_data

array([455.25      , 469.        , 466.3500061 , 463.8999939 ,
       461.79998779, 466.70001221, 445.8999939 , 445.5       ,
       457.54998779, 460.54998779, 453.8500061 , 432.04998779,
       426.8999939 , 432.95001221, 439.3500061 , 436.5       ,
       429.95001221,          nan])

In [62]:
comp_data=comp_data
comp_data

array([455.25      , 469.        , 466.3500061 , 463.8999939 ,
       461.79998779, 466.70001221, 445.8999939 , 445.5       ,
       457.54998779, 460.54998779, 453.8500061 , 432.04998779,
       426.8999939 , 432.95001221, 439.3500061 , 436.5       ,
       429.95001221,          nan])

In [47]:
log = [np.log10(comp_data[i]/comp_data[i-1]) for i in range(1,17)]

In [48]:
log

[-0.0069348872887404454,
 -0.00011751109376168259,
 -0.009324668850229015,
 0.008560124622679023,
 0.006079067242481015,
 -0.0012788834751178897,
 0.0001746070440867717,
 -0.00515140986655492,
 -0.016136836417933998,
 0.0021946650474120285,
 -0.030280435540689068,
 0.004282017824660357,
 0.017028350087338873,
 -0.009666708053558253,
 0.005424793759656438,
 0.0024379467868496712]

In [36]:
std =np.std(log)
std

0.006245148721005063